In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRU, TimeDistributed
import pickle

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Text_Gen/tokenizer.pickle', 'rb') as f:
  tokenizer = pickle.load(f)

In [ ]:
num_char = len(tokenizer.word_index)
print(num_char)

In [ ]:
word_index = tokenizer.word_index
word_index = dict((item, key) for key, item in word_index.items())

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Text_Gen/Checkpoint')

In [ ]:
model.summary()

In [ ]:
def preprocess(text): # can only handle text sequences of the same length
  X = np.array(tokenizer.texts_to_sequences(text)) - 1
  return tf.one_hot(X, num_char)

In [ ]:
def predict_next_char(text, temp = 1): 
  X = tf.one_hot(np.array(tokenizer.texts_to_sequences([text])) - 1, num_char)
  probas = model.predict(X)[0,-1:,:]
  rescaled_logits = tf.math.log(probas) / temp
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return word_index[int(char_id.numpy())]


In [ ]:
predict_next_char('I went to the ball with my famil')

In [ ]:
def generate_text(seed_text, n_chars = 100, temp = 1):
  for i in range(n_chars):
    seed_text += predict_next_char(seed_text, temp)
  print(seed_text)

In [ ]:
generate_text('I went to the ball with my family', temp = 2)

In [ ]:
generate_text('I went to the ball with my family', temp = 0.1)

In [ ]:
generate_text('I went to the ball with my family', temp = 1)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Text_Gen/Train_corp.txt', 'r') as f:
  train_text = f.read()
with open('/content/drive/MyDrive/Colab Notebooks/Text_Gen/Valid_corp.txt', 'r') as f:
  valid_text = f.read()

In [ ]:
[encoded_train] = np.array(tokenizer.texts_to_sequences([train_text])) - 1
[encoded_valid] = np.array(tokenizer.texts_to_sequences([valid_text])) - 1

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(encoded_train)
valid_data = tf.data.Dataset.from_tensor_slices(encoded_valid)

In [ ]:
def window_dataset(dataset, num_char, seq_len = 100, batch_size = 128, shuff_buff = 10000000):

  window_length = seq_len + 1 # target = input shifted 1 character ahead
  #create windows of length seq_len + 1; use seq_len to predict last char
  dataset = dataset.window(window_length, shift=1, drop_remainder=True)
  # we have a dataset of datasets so we need to flatten i
  dataset = dataset.flat_map(lambda window: window.batch(window_length))

  dataset = dataset.shuffle(shuff_buff).batch(batch_size)
  dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

  dataset = dataset.map(
  lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=num_char),
  Y_batch))
  dataset = dataset.prefetch(1)
  return dataset

In [ ]:
T = window_dataset(train_data, num_char = num_char)
V = window_dataset(valid_data, num_char = num_char)

In [ ]:
check_point = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Text_Gen/Checkpoint', 
          monitor = 'val_loss', save_weights_only = False, save_freq = 1000,
          save_best_only = False, verbose = 1)

In [ ]:
history = model.fit(T, epochs=10, validation_data = V, callbacks = [check_point])